In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from keras import optimizers
from keras import regularizers

# Define the search space
param_space = {
    'units': [3, 4],
    'optimizer': ['adam', 'sgd'],
    'learning_rate': [0.06, 0.05, 0.07, 0.08, 0.1],
    'batch_size': [16, 32, 64],
    'epochs': [50, 60, 70, 80],
    'weight_decay': [0.001, 0.01, 0.002, 0.003],
    'momentum': [0.7, 0.8, 0.9]
}

# Number of random combinations to try
num_trials = 15

for _ in range(num_trials):
    # Randomly sample from the search space
    params = {
        'units': np.random.choice(param_space['units']),
        'optimizer': np.random.choice(param_space['optimizer']),
        'learning_rate': np.random.choice(param_space['learning_rate']),
        'batch_size': np.random.choice(param_space['batch_size']),
        'epochs': np.random.choice(param_space['epochs']),
        'weight_decay': np.random.choice(param_space['weight_decay']),
        'momentum': np.random.choice(param_space['momentum'])
        
    }

    # Build the Keras model
    model = Sequential()
    model.add(Dense(units=params['units'], input_shape=(6,5), activation='relu', kernel_regularizer=regularizers.l2(params['weight_decay'])))
    model.add(Dense(units=1, activation='sigmoid'))
    
    # Set the optimizer with the sampled learning rate
    if params['optimizer'] == 'adam':
        optimizer = optimizers.Adam(learning_rate=params['learning_rate'])
    elif params['optimizer'] == 'sgd':
        optimizer = optimizers.SGD(learning_rate=params['learning_rate'], momentum=params['momentum'])
    else:
        raise ValueError("Unsupported optimizer")

    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    # Train the model with early stopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    history = model.fit(X_one_hot, y, epochs=params['epochs'],
                         batch_size=params['batch_size'], 
                         validation_split=0.2, 
                         callbacks=[early_stopping],
                           verbose=0)

    # Evaluate the model
    loss, accuracy = model.evaluate(X_one_hot, y, verbose=0)

    print(f"Trial {_:>2}: Units={params['units']}, Momentum={params['momentum']}, Epochs={params['epochs']}, Decay={params['weight_decay']}, Optimizer={params['optimizer']}, Learning Rate={params['learning_rate']:.3f}, Batch Size={params['batch_size']}, Accuracy={accuracy:.4f}")

